In [1]:
import cache
import meta
import meta_session

import nept
import numpy as np
import pandas as pd
import scipy.stats
import scipy.signal
from shapely.geometry import LineString, Point
from shapely.ops import split
import statsmodels.api as sm
import matplotlib.pyplot as plt

from utils import save_ttest_results

%matplotlib inline

In [2]:
info = meta_session.r063d6
group = "combined"

def get(key, info=None, group=None):
    if info is not None:
        return cache.load(f"ind-{info.session_id}", key)
    return cache.load(f"grp-{group}", key)

In [5]:
swr_rate_byphase_restonly = get("swr_rate_byphase_restonly", group=group)

In [6]:
swr_rate_byphase_restonly

{'prerecord': [0.1244719769081249,
  0.04888990866719722,
  0.1423433978052302,
  0.12372936812786428,
  0.015794622024388902,
  0.22076995139528652,
  0.23554846380199235,
  0.054625337917307525,
  0.09499156111933894,
  0.12554328151600996,
  0.2121594659544107,
  0.1884777265235031,
  0.18624152349462952,
  0.09508165964732618,
  0.07896415741575524,
  0.16767117929306258,
  0.10211358646564933,
  0.07973280093835004,
  0.077526455529161,
  0.09489153904678499,
  0.05793203229875594,
  0.12307723361025104,
  0.07421676212142118,
  0.16041355721349973,
  0.10486255148928227,
  0.1328312628915158,
  0.1242709240633338],
 'phase1': [0.03547786911367943,
  0.060534663130114956,
  0.0275843216508442,
  0.043157947941181864,
  0.0,
  0.06083543107418651,
  0.15316647434346742,
  0.02172319379608843,
  0.019857742902814987,
  0.018341636194636992,
  0.03311699984355581,
  0.031966705525394,
  0.02562326762155204,
  0.0,
  0.013249179316022599,
  0.10577788570570124,
  0.0,
  0.036885307956

In [ ]:
swr_n_byphase = get("swr_n_byphase", group=group)
replay_n_byphase = get("replay_n_byphase", group=group)

swr_n_byexperience = get("swr_n_byexperience", group=group)
replay_n_byexperience = get("replay_n_byexperience", group=group)

In [ ]:
# replay prop over task time for u and shortcut
alpha = 0.05
pairs = [("pauseA", "pauseB"), ("prerecord", "postrecord"), ("phase1", "phase2"), ("phase2", "phase3")]

for trajectory in meta.trajectories:
    print(trajectory)
    for pair in pairs:
        n1 = swr_n_byphase[pair[0]]
        p1 = replay_n_byphase[trajectory][pair[0]] / swr_n_byphase[pair[0]]

        n2 = swr_n_byphase[pair[1]]
        p2 = replay_n_byphase[trajectory][pair[1]] / swr_n_byphase[pair[1]]

        population1 = np.random.binomial(1, p1, n1)
        population2 = np.random.binomial(1, p2, n2)
        t_stat, p_val, n = sm.stats.ttest_ind(population1, population2)
        print(pair, p_val, p_val<alpha)

In [ ]:
# replay prop btw u, shortcut for each task_time
for task_time in meta.task_times:
    n1 = swr_n_byphase[task_time]
    p1 = replay_n_byphase["u"][task_time] / swr_n_byphase[task_time]

    n2 = swr_n_byphase[task_time]
    p2 = replay_n_byphase["full_shortcut"][task_time] / swr_n_byphase[task_time]

    population1 = np.random.binomial(1, p1, n1)
    population2 = np.random.binomial(1, p2, n2)
    t_stat, p_val, n = sm.stats.ttest_ind(population1, population2)
    print(task_time, p_val, p_val<alpha)

In [ ]:
# replay prop btw u, shortcut for each experience time
for task_time in meta.experiences:
    n1 = swr_n_byexperience[task_time]
    p1 = replay_n_byexperience["only_u"][task_time] / swr_n_byexperience[task_time]

    n2 = swr_n_byexperience[task_time]
    p2 = replay_n_byexperience["only_full_shortcut"][task_time] / swr_n_byexperience[task_time]

    population1 = np.random.binomial(1, p1, n1)
    population2 = np.random.binomial(1, p2, n2)
    t_stat, p_val, n = sm.stats.ttest_ind(population1, population2)
    print(task_time, p_val, p_val<alpha)